In [37]:
import pandas as pd
import numpy as np
import glob
import cv2
from sklearn.preprocessing import LabelBinarizer
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, SeparableConv2D, Convolution2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

In [16]:
!pip install opencv-python


     |████████████████████████████████| 49.1 MB 9.1 MB/s eta 0:00:011    |███████▌                        | 11.4 MB 7.2 MB/s eta 0:00:06     |████████████████                | 24.6 MB 12.5 MB/s eta 0:00:02     |███████████████████████████▍    | 42.0 MB 12.8 MB/s eta 0:00:01


In [19]:
train_normal_imgs = glob.glob ("chest_xray/train/NORMAL/*")
train_pneumonia_imgs = glob.glob ("chest_xray/train/PNEUMONIA/*")
test_normal_imgs = glob.glob ("chest_xray/test/NORMAL/*")
test_pneumonia_imgs = glob.glob ("chest_xray/test/PNEUMONIA/*")

In [39]:
len(train_pneumonia_imgs)

3875

In [25]:

train_normal_data = []
train_pneumonia_data = []
test_normal_data = []
test_pneumonia_data = []
X_train = []
X_test = []
y_train = []
y_test = []

normal_data = []
covid_imgs = glob.glob ("/content/datasets/covid/*")
normal_imgs = glob.glob("/content/datasets/healthy/*")


label = "positive"
for img in train_pneumonia_imgs:
    img = cv2.imread(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (225, 225))
    X_train.append(img)
    y_train.append(label)
print('X_train shape:', np.array(X_train).shape)
print('y_train shape:', np.array(y_train).shape)


label = "negative"
for img in train_normal_imgs:
    img = cv2.imread(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (225, 225))
    X_train.append(img)
    y_train.append(label)
print('X_train shape:', np.array(X_train).shape)
print('y_train shape:', np.array(y_train).shape)


label = "positive"
for img in test_pneumonia_imgs:
    img = cv2.imread(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (225, 225))
    X_test.append(img)
    y_test.append(label)
print('X_test shape:', np.array(X_test).shape)
print('y_test shape:', np.array(y_test).shape)   

label = "negative"
for img in test_normal_imgs:
    img = cv2.imread(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (225, 225))
    X_test.append(img)
    y_test.append(label)
print('X_test shape:', np.array(X_test).shape)
print('y_test shape:', np.array(y_test).shape)   

X_train = np.array(X_train) / 255.0
X_test = np.array(X_test) / 255.0
y_train = np.array(y_train)
y_test = np.array(y_test)

# One Hot Encode the data
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
lb = LabelBinarizer()
y_test = lb.fit_transform(y_test)


X_train shape: (3875, 225, 225, 3)
y_train shape: (3875,)
X_train shape: (5216, 225, 225, 3)
y_train shape: (5216,)
X_test shape: (390, 225, 225, 3)
y_test shape: (390,)
X_test shape: (624, 225, 225, 3)
y_test shape: (624,)


In [26]:
y_train

array([[1],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [41]:
#Initializing the CNN
classifier = Sequential()

In [42]:
# Step1: Convolution
classifier.add(Convolution2D(32, 3, 3, input_shape=(225, 225, 3), activation='relu'))

# Step2: Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))

# Step3: Faltenning
classifier.add(Flatten())

# Step4: Full connection
classifier.add(Dense(output_dim = 128, activation='relu'))
classifier.add(Dense(output_dim = 1, activation='sigmoid'))

# Step5: Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Step6: Fitting the CNN to the images
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        'chest_xray/train',
        target_size=(225, 225),
        batch_size=32,
        class_mode='binary')

label_map = training_set.class_indices

test_set = test_datagen.flow_from_directory(
        'chest_xray/test',
        target_size=(225, 225),
        batch_size=32,
        class_mode='binary')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(225, 225,..., activation="relu")`
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  # This is added back by InteractiveShellApp.init_path()
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1)`
  if sys.path[0] == '':


Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [45]:


classifier.fit_generator(
        training_set,
        steps_per_epoch=5216,
        epochs=5,
        validation_data=test_set,
        validation_steps=800)

Epoch 1/5
5216/5216 [==============================] - 7995s 2s/step - loss: 0.1020 - accuracy: 0.9620 - val_loss: 0.5339 - val_accuracy: 0.9135
Epoch 2/5
5216/5216 [==============================] - 7696s 1s/step - loss: 0.0752 - accuracy: 0.9721 - val_loss: 0.0101 - val_accuracy: 0.9167
Epoch 3/5
5216/5216 [==============================] - 7053s 1s/step - loss: 0.0586 - accuracy: 0.9787 - val_loss: 0.8162 - val_accuracy: 0.9135
Epoch 4/5
5216/5216 [==============================] - 7351s 1s/step - loss: 0.0489 - accuracy: 0.9824 - val_loss: 0.0678 - val_accuracy: 0.9071
Epoch 5/5
5216/5216 [==============================] - 6968s 1s/step - loss: 0.0418 - accuracy: 0.9850 - val_loss: 1.7572 - val_accuracy: 0.9006


In [46]:
type(classifier)

keras.engine.sequential.Sequential

In [47]:
test_gen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test = test_gen.flow_from_directory(
        'chest_xray/te',
        target_size=(225, 225),
        class_mode='binary')
classifier.predict(test)



Found 0 images belonging to 0 classes.


[]

In [165]:
classifier.save('COVID19-CNN.model')


In [49]:
test_covid_img = glob.glob ("chest_xray/te/NORMAL/*")
normal_imgs = glob.glob("/content/datasets/healthy/*")


# label = "positive"
# for img in train_pneumonia_imgs:
#     img = cv2.imread(img)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     img = cv2.resize(img, (225, 225))
neg_img = cv2.imread('chest_xray/te/neg.jpeg')
neg_img = cv2.resize(neg_img, (225, 225))

pos_img = cv2.imread('chest_xray/te/pos.jpeg')
pos_img = cv2.resize(pos_img, (225, 225))

In [163]:

CATEGORIES = ['NORMAL','PNEUMONIA']  # will use this to convert prediction num to string value

IMG_SIZE = 225

def prepare(filepath):
    IMG_SIZE = 225
    img_array = cv2.imread(filepath)  # read in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)  # return the image with shaping that TF wants.


prediction = classifier.predict([prepare('chest_xray/te/neg10.jpeg')])

In [164]:
CATEGORIES[int(prediction[0][0])]

'NORMAL'